In [1]:
import tensorflow as tf
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
import pandas as pd
import numpy as np
import os
import torch
import gradio as gr

In [199]:
#Read Dataset and Cleanin Process
df = pd.read_csv(r"train.csv")
df = np.array_split(df,1000)[2]
df

C:\Users\ale11\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


,Conversation
214,The conversation between human and AI assistan...
215,The conversation between human and AI assistan...
216,The conversation between human and AI assistan...
217,The conversation between human and AI assistan...
218,The conversation between human and AI assistan...
...,...
316,The conversation between human and AI assistan...
317,The conversation between human and AI assistan...
318,The conversation between human and AI assistan...
319,The conversation between human and AI assistan...


In [200]:
df['Conversation1']=df['Conversation'].str.replace('\n', '')

In [203]:
df['Conversation1']=df['Conversation1'].str.replace('The conversation between human and AI assistant.[|Human|] ','')

In [205]:
df['Human']="User: "+df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[0]
df['AI']="ChatBot: "+df['Conversation1'].str.split("\[\|AI\|\]",expand=True)[1]

In [207]:
df['Whole']= df['Conversation1'].str.replace('[|AI|]', '')

In [209]:
df_train=df['Whole']

In [19]:
#tokenize the model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

C:\Users\ale11\anaconda3\lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
# Creating Structure for DataLoader of Torch
class ChatDataset(Dataset):
    def __init__(self, input_encodings, target_encodings):
        self.input_encodings = input_encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.input_encodings['input_ids'])

    def __getitem__(self, idx):
        input_ids = self.input_encodings['input_ids'][idx]
        attention_mask = self.input_encodings['attention_mask'][idx]
        labels = self.target_encodings['input_ids'][idx]
        return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}

In [211]:
# Tokenizing the inputs and labels Create dataset and dataloader

input_encodings = tokenizer(list(df['Human']), padding=True, truncation=True, return_tensors="pt")
target_encodings = tokenizer(list(df['AI']), padding=True, truncation=True, return_tensors="pt")

dataset = ChatDataset(input_encodings, target_encodings)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [73]:
#Define optimizer and Train the model
optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()
for epoch in range(5):
    for batch in dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch {epoch}, Loss: {loss.item()}')

C:\Users\ale11\anaconda3\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 0, Loss: 0.8559985160827637
Epoch 0, Loss: 1.4361222982406616
Epoch 0, Loss: 0.751570999622345
Epoch 0, Loss: 1.0553346872329712
Epoch 0, Loss: 1.6190959215164185
Epoch 0, Loss: 0.8255897164344788
Epoch 0, Loss: 0.9983565211296082
Epoch 0, Loss: 1.1314153671264648
Epoch 0, Loss: 1.5597975254058838
Epoch 0, Loss: 0.789936900138855
Epoch 0, Loss: 0.8923855423927307
Epoch 0, Loss: 2.143014430999756
Epoch 0, Loss: 0.9085750579833984
Epoch 0, Loss: 1.1713602542877197
Epoch 0, Loss: 1.2147481441497803
Epoch 0, Loss: 1.0133049488067627
Epoch 0, Loss: 1.1518000364303589
Epoch 0, Loss: 0.9206996560096741
Epoch 0, Loss: 0.828273355960846
Epoch 0, Loss: 1.0379703044891357
Epoch 0, Loss: 1.5836138725280762
Epoch 0, Loss: 1.686095118522644
Epoch 0, Loss: 1.3690797090530396
Epoch 0, Loss: 0.5834330320358276
Epoch 0, Loss: 1.1401267051696777
Epoch 0, Loss: 0.8470251560211182
Epoch 0, Loss: 1.451306939125061
Epoch 0, Loss: 1.1547185182571411
Epoch 0, Loss: 1.5060133934020996
Epoch 0, Loss: 2.850

In [225]:
# Saving the model

# Define a checkpoint directory
checkpoint_dir = './T5Checkpoint'

# Ensure the directory exists
os.makedirs(checkpoint_dir, exist_ok=True)

# Save model checkpoint with unique filename (including epoch number)
model.save_pretrained(os.path.join(checkpoint_dir, f'model_epoch_{epoch}'))

# Save the optimizer state with unique filename
torch.save(optimizer.state_dict(), os.path.join(checkpoint_dir, f'optimizer_epoch_{epoch}.pt'))

# Save tokenizer
tokenizer.save_pretrained(os.path.join(checkpoint_dir, f'tokenizer_epoch_{epoch}'))

('./T5Checkpoint\\tokenizer_epoch_22\\tokenizer_config.json',
 './T5Checkpoint\\tokenizer_epoch_22\\special_tokens_map.json',
 './T5Checkpoint\\tokenizer_epoch_22\\spiece.model',
 './T5Checkpoint\\tokenizer_epoch_22\\added_tokens.json')

In [7]:
#Loading the model

# Get the latest checkpoint directory
checkpoint_dir = './T5Checkpoint'
latest_epoch = max([int(folder.split('_')[2]) for folder in os.listdir(checkpoint_dir) if 'model_epoch' in folder])

# Load the model from the latest checkpoint
model = T5ForConditionalGeneration.from_pretrained(os.path.join(checkpoint_dir, f'model_epoch_{latest_epoch}'))

# Load optimizer state
optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer.load_state_dict(torch.load(os.path.join(checkpoint_dir, f'optimizer_epoch_{latest_epoch}.pt')))

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained(os.path.join(checkpoint_dir, f'tokenizer_epoch_{latest_epoch}'))

C:\Users\ale11\anaconda3\lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
C:\Users\ale11\AppData\Local\Temp\ipykernel_4684\463555961.py:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user v

In [217]:
#Train the model for back and fort training

model.train()
for epoch in range(latest_epoch+1, latest_epoch+3):
    for batch in dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 21, Loss: 2.1283113956451416
Epoch 21, Loss: 1.217342734336853
Epoch 21, Loss: 1.5536366701126099
Epoch 21, Loss: 2.498788356781006
Epoch 21, Loss: 2.5498788356781006
Epoch 21, Loss: 2.1012587547302246
Epoch 21, Loss: 1.5909079313278198
Epoch 21, Loss: 2.1831471920013428
Epoch 21, Loss: 1.5769462585449219
Epoch 21, Loss: 1.8055353164672852
Epoch 21, Loss: 2.8687052726745605
Epoch 21, Loss: 2.0393266677856445
Epoch 21, Loss: 2.7164273262023926
Epoch 21, Loss: 1.61953604221344
Epoch 21, Loss: 1.4889483451843262
Epoch 21, Loss: 1.8661705255508423
Epoch 21, Loss: 1.649787187576294
Epoch 21, Loss: 2.460588216781616
Epoch 21, Loss: 2.0209157466888428
Epoch 21, Loss: 1.6800830364227295
Epoch 21, Loss: 2.3686745166778564
Epoch 21, Loss: 2.2393393516540527
Epoch 21, Loss: 1.6449662446975708
Epoch 21, Loss: 1.2595983743667603
Epoch 21, Loss: 1.4962043762207031
Epoch 21, Loss: 3.001206398010254
Epoch 21, Loss: 1.683516263961792
Epoch 21, Loss: 1.7339421510696411
Epoch 21, Loss: 2.8903803825

In [11]:
# Function to train the Bot
def chat_with_bot(input_text):
    # Add task prefix
    model.eval()
    input_text = f"User: {input_text}"
    input_ids = tokenizer.encode(input_text, return_tensors='pt')
    print(input_ids)
    output_ids = model.generate(input_ids, max_length=300)
    print(output_ids)
    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return response

response = chat_with_bot("Hi.My names Ahdieh.I m from a small city in Iran.My father had a heart attack on sunday as doctor said cpr condition.after 40 minuts he came back to life. now he isnot conscious.he is in ICU.his doctor said his conscious rate is 5.what do you think about his condition?If it is needed I can send you his cardiograf and blood test and the drugs they are using for him in ICU.Kindly tell me can we carry him to a better hospital in Tehran(It takes 4 hours to reach there)?")
print(response)

tensor([[ 6674,    10,  2018,     5,  7008,  3056,  9710,  2498,   107,     5,
           196,     3,    51,    45,     3,     9,   422,   690,    16,  7449,
             5,  7008,  2353,   141,     3,     9,   842,  3211,    30,  1997,
          1135,    38,  2472,   243,     3,    75,   102,    52,  1706,     5,
         10245,  1283, 21167,     7,     3,    88,   764,   223,    12,   280,
             5,   230,     3,    88,    19,  2264, 13381,     5,    88,    19,
            16,    27,  5211,     5, 10193,  2472,   243,   112, 13381,  1080,
            19,  3594,  9170,   103,    25,   317,    81,   112,  1706,    58,
          5801,    34,    19,   906,    27,    54,  1299,    25,   112, 16216,
          8290,    11,  1717,   794,    11,     8,  4845,    79,    33,   338,
            21,   376,    16,    27,  5211,     5,   439,    77,    26,   120,
           817,   140,    54,    62,  2331,   376,    12,     3,     9,   394,
          2833,    16, 31238,   599,   196,    17,  

In [9]:
# Function for Gradio
def predict_disease(input, history):
    return  chat_with_bot(input)

In [13]:
# Gradio Interface
interface = gr.ChatInterface(
    fn=predict_disease,
    title="DocRoboto",
    description="Predict diseases based on symptoms entered by the user. CAUTION: This is not a real doctor....This is a tool and not a replacement for you primary care physican.",
    theme="compact", 
)

interface.launch(share=True, inline=True)

C:\Users\ale11\anaconda3\lib\site-packages\gradio\blocks.py:982: UserWarning: Cannot load compact. Caught Exception: module 'huggingface_hub.utils' has no attribute '_errors'
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://5eb20e2b3110b083a0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
